In [1]:
import yaml

In [5]:
with open('openmeteo.yml') as f:
    openapispec = yaml.safe_load(f)

In [7]:
openapispec.keys()

dict_keys(['openapi', 'info', 'paths'])

In [86]:
import json

In [150]:
class OpenAPISpec:
    
    def __init__(self, spec_dict: dict):
        from openapi_spec_validator import validate_spec
        validate_spec(spec_dict)
        self.spec_dict = spec_dict
        
    @classmethod
    def from_filename(cls, filename: str):
        from openapi_spec_validator.readers import read_from_filename

        spec_dict, _ = read_from_filename(filename)
        return cls(spec_dict)
    
    def get_paths(self, *args):
        paths = []
        for p, v in self.spec_dict['paths'].items():
            _strs = [f"Name: {p}"]
            if "description" in v['get']:
                _strs.append(f"Description: {v['get']['description']}")
            if "servers" in v:
                _strs.append(f"Servers: {v['servers']}")
            _str = "\n".join(_strs)
            paths.append(_str)
        return "\n\n".join(paths)
    
    def get_parameters(self, path: str):
        params = self.spec_dict['paths'][path]['get']['parameters']
        return ", ".join([p['name'] for p in params])
    
    def get_param_spec(self, path_param_string: str):
        json_string = json.loads(path_param_string)
        path, param = json_string['path'], json_string['parameter_name']
        params = self.spec_dict['paths'][path]['get']['parameters']
        param_dict = {p['name']: p for p in params}
        return str(param_dict[param])

In [151]:
spec.spec_dict['paths']['/v1/fish/{fishID}']['get']['description']

'Get fish data in JSON format. \n\nThe {fishID} parameter is optional and can be removed to get all the fish data.'

In [152]:
spec = OpenAPISpec.from_filename('/Users/harrisonchase/Downloads/swagger.json')

In [204]:
from openapi3 import OpenAPI
import yaml

# load the spec file and read the yaml
with open('/Users/harrisonchase/Downloads/swagger.json') as f:
    spec = yaml.safe_load(f.read())

# parse the spec into python - this will raise if the spec is invalid
api = OpenAPI(spec)

In [ ]:
def deslotify(obj):
    if not hasattr(obj, "__slots__"):
        return obj
    return {s: deslotify(getattr(obj, s)) for s in obj.__slots__ if hasattr(obj, s) and getattr(obj, s) is not None}
    

In [194]:
"foo".__slots__

AttributeError: 'str' object has no attribute '__slots__'

In [202]:
obj = api.paths['/v1/fish/{fishID}']
obj.get.description

'Get fish data in JSON format. \n\nThe {fishID} parameter is optional and can be removed to get all the fish data.'

In [153]:
spec.spec_dict['paths']

{'/v1/fish/{fishID}': {'parameters': [{'schema': {'type': 'string'},
    'name': 'fishID',
    'in': 'path',
    'description': "Either the file-name of a fish or it's ID.",
    'required': True}],
  'get': {'summary': 'Fish',
   'tags': ['Fish'],
   'responses': {'200': {'description': 'OK',
     'content': {'application/json': {'schema': {'type': 'object',
        'properties': {'id': {'type': 'integer',
          'description': 'Number of the fish (as found in the critterpedia).'},
         'file-name': {'type': 'string',
          'description': 'Name of the image files.'},
         'name': {'type': 'object',
          'description': 'Name of the fish in different languages.',
          'required': ['name-USen',
           'name-CNzh',
           'name-EUde',
           'name-EUes',
           'name-EUfr',
           'name-EUit',
           'name-JPja',
           'name-KRko',
           'name-EUnl',
           'name-EUru',
           'name-EUen',
           'name-USes',
          

In [154]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain import OpenAI, LLMChain

In [169]:
tools = [
    Tool(
        name = "List Paths",
        func=spec.get_paths,
        description="Can be used to list all available paths for this API spec. Input should be an empty string (it's not used)."
    ),
    Tool(
        name = "Get Params",
        func=spec.get_parameters,
        description="Get parameters for a given path. The input should be exactly ones of the paths available to this API spec, including any parameter placeholders."
    ),
    Tool(
        name = "Get Param Spec",
        func=spec.get_param_spec,
        description='Get specification for a particular parameter for a particular path. The input should be valid json in the format `{{"path": path, "parameter_name": parameter_name}}` where "path" is a path available to this API spec and "parameter_name" is a parameter available for this path.'
    ),
    
]

In [170]:
prefix = """You are an agent designed to interact with a particular API.
Your goal is to return a final answer of a particular URL with parameters that can be called to answer a particular question.
You have access to the following tools which help you learn more about the API you are interacting with.
Only use the below tools. Only use the information returned by the below tools to construct your final answer."""
suffix = """Begin!"

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=["input", "agent_scratchpad"]
)

In [171]:
print(prompt.template)

You are an agent designed to interact with a particular API.
Your goal is to return a final answer of a particular URL with parameters that can be called to answer a particular question.
You have access to the following tools which help you learn more about the API you are interacting with.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.

List Paths: Can be used to list all available paths for this API spec. Input should be an empty string (it's not used).
Get Params: Get parameters for a given path. The input should be exactly ones of the paths available to this API spec, including any parameter placeholders.
Get Param Spec: Get specification for a particular parameter for a particular path. The input should be valid json in the format `{{"path": path, "parameter_name": parameter_name}}` where "path" is a path available to this API spec and "parameter_name" is a parameter available for this path.

Use the following format:

In [172]:
llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)

In [173]:
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)

In [174]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [175]:
agent_executor.run("How rare is a barracuda?")



> Entering new AgentExecutor chain...
Thought: I need to find out what parameters I need to call the API
Action: List Paths
Action Input: ""
Observation: Name: /v1/fish/{fishID}
Description: Get fish data in JSON format. 

The {fishID} parameter is optional and can be removed to get all the fish data.

Name: /v1/sea/{seaID}
Description: Get sea creature data in JSON format. 

The {seaID} parameter is optional and can be removed to get all the sea creature data.

Name: /v1/bugs/{bugID}
Description: Get bugs data in JSON format. 

The {bugID} parameter is optional and can be removed to get all the bugs data.

Name: /v1/fossils/{fossilName}
Description: Get fossil data in JSON format. 

The {fossilName} parameter is optional and can be removed to get all the fossils data.

Name: /v1/villagers/{villagerID}
Description: Get villager data in JSON format. 

The {villagerID} parameter is optional and can be removed to get all the villagers data.

Name: /v1/icons/fish/{fishID}
Description: Ge

KeyError: '/v1/fish/'

In [149]:
%debug

> /var/folders/bm/ylzhm36n075cslb9fvvbgq640000gn/T/ipykernel_405/293112481.py(18)get_paths()
     16         paths = []
     17         for p, v in self.spec_dict['paths'].items():
---> 18             _str = f"Name: {p}\nDescription: {v['get']['description']}"
     19             paths.append(_str)
     20         return "\n\n".join(paths)

ipdb> v
{'parameters': [{'schema': {'type': 'string'}, 'name': 'wallmountedID', 'in': 'path', 'required': True}], 'get': {'summary': 'Wallmounted', 'tags': ['Items'], 'responses': {'200': {'description': 'OK'}, '404': {'description': 'Not Found'}}}}
ipdb> p
*** SyntaxError: unexpected EOF while parsing
ipdb> u
> /Users/harrisonchase/workplace/langchain/langchain/agents/agent.py(418)_call()
    416                 try:
    417                     # We then call the tool on the tool input to get an observation
--> 418                     observation = tool.func(output.tool_input)
    419                     color = color_mapping[output.tool]
    420  

In [68]:
print(spec.get_paths())

Name: /v1/forecast
Description: 7 day weather variables in hourly and daily resolution for given WGS84 latitude and longitude coordinates. Available worldwide.


In [63]:
spec.get_parameters("/v1/forecast")

'hourly, daily, latitude, longitude, current_weather, temperature_unit, windspeed_unit, timeformat, timezone, past_days'

In [64]:
spec.get_param_spec("/v1/forecast, current_weather")

"{'name': 'current_weather', 'in': 'query', 'schema': {'type': 'boolean'}}"

In [47]:
spec_dict['paths']['/v1/forecast']['get']['parameters'][6]

{'name': 'windspeed_unit',
 'in': 'query',
 'schema': {'type': 'string',
  'default': 'kmh',
  'enum': ['kmh', 'ms', 'mph', 'kn']}}

In [229]:
import re
def parse_items(text):
    _res = re.findall(r'\[.*?\]', text)
    res = []
    for r in _res:
        val = r[1:-1]
        if val[0] != '"':
            val = int(val)
        else:
            val = val[1:-1]
        res.append(val)
    return res

In [300]:
class JsonObj:
    
    def __init__(self, _dict: dict):
        self.dict = _dict
        
    def keys(self, text: str):
        try:
            items = parse_items(text)
            val = self.dict
            for i in items:
                val = val[i]
            return str(val.keys())
        except Exception as e:
            return repr(e)
    
    def value(self, text: str):
        try:
            items = parse_items(text)
            val = self.dict
            for i in items:
                val = val[i]
            
            if isinstance(val, dict) and len(str(val)) > 200:
                return "Value is a large dictionary, should explore its keys directly"
            val = str(val)
            if len(val) > 200:
                val = val[:200] + "..."
            return val
        except Exception as e:
            return repr(e)

In [301]:
obj = JsonObj(spec)

In [302]:
obj.value('data')

'Value is a large dictionary, should explore its keys directly'

In [303]:
tools = [
    Tool(
        name = "List Keys",
        func=obj.keys,
        description="Can be used to list all keys. Before calling this you should be SURE that the path to this exists."
    ),
    Tool(
        name = "See Value",
        func=obj.value,
        description="Can be used to see stringified value. Before calling this you should be SURE that the path to this exists."
    ),
    
]

In [304]:
prefix = """You are an agent designed to interact with JSON.
Your goal is to return a final answer by interacting with the JSON.
You have access to the following tools which help you learn more about the JSON you are interacting with.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
Your input to the tools should be in the form of `data["key"][0]` where `data` is the JSON blob you are interacting with, and the syntax used is Python. 
You should only use keys that you know for a fact exist. You must validate that a key exists by seeing it previously when calling `List Keys`. If you have not seen a key in one of those responses, you cannot use it."""
suffix = """Begin!"

Question: {input}
Thought: I should look at the keys that exist in data to see what I have access to
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=["input", "agent_scratchpad"]
)

In [305]:
llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)

In [306]:
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)

In [307]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [309]:
agent_executor.run("what does the bugs endpoint do?")



> Entering new AgentExecutor chain...
Action: List Keys
Action Input: data
Observation: dict_keys(['openapi', 'info', 'servers', 'paths', 'components', 'security'])
Thought: I should look at the paths key to see what endpoints exist
Action: List Keys
Action Input: data["paths"]
Observation: dict_keys(['/v1/fish/{fishID}', '/v1/sea/{seaID}', '/v1/bugs/{bugID}', '/v1/fossils/{fossilName}', '/v1/villagers/{villagerID}', '/v1/icons/fish/{fishID}', '/v1/icons/bugs/{bugID}', '/v1/icons/villagers/{villagerID}', '/v1/images/fish/{fishID}', '/v1/images/bugs/{bugID}', '/v1/images/villagers/{villagerID}', '/v1/images/fossils/{fossilName}', '/v1/songs/{songID}', '/v1/images/songs/{songID}', '/v1/music/{songID}', '/v1/art/{artID}', '/v1/icons/art/{artID}', '/v1/hourly/{songID}', '/v1/backgroundmusic/{songID}', '/v1/houseware/{housewareID}', '/v1/wallmounted/{wallmountedID}', '/v1/misc/{miscID}', '/v1/images/funiture/{furnitureFileName}'])
Thought: I should look at the /v1/bugs/{bugID} endpoint to

'The /v1/bugs/{bugID} endpoint returns all the bugs data in JSON format. The {bugID} parameter is optional and can be removed to get all the bugs data.'